# Preprcessing

In [23]:
import tweepy
import configparser
import pandas as pd
import numpy as np
import re
import pickle
import nltk
import string
import collections
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder

In [24]:
# read data 
train_data_file = open('project-data/train.data.txt', 'r')
train_lines = train_data_file.readlines()
train_events =[]
# Strips the newline character
for line in train_lines:
    train_events.append(list(map(int,line.strip('\n').split(','))))
    
train_label_file = open('project-data/train.label.txt', 'r') 
train_labels = train_label_file.readlines()
train_labels = [label.strip('\n') for label in train_labels]


dev_data_file = open('project-data/dev.data.txt', 'r')
dev_lines = dev_data_file.readlines()
dev_events =[]
# Strips the newline character
for line in dev_lines:
    dev_events.append(list(map(int,line.strip('\n').split(','))))
    
dev_label_file = open('project-data/dev.label.txt', 'r')
dev_labels = dev_label_file.readlines()
dev_labels = [label.strip('\n') for label in dev_labels]

In [50]:
# open train text file
f = open(f'./tweet_text.pckl','rb')
train_data = pickle.load(f)
f.close()


# open dev text file
f = open(f'./dev_tweet_text.pckl','rb')
dev_data = pickle.load(f)
f.close()

In [53]:
def clean_text(text):
    text = re.sub(r'@[A-Za-z0-9]+','',text) #remove @mention
    text = re.sub(r'#','',text) # remove the hashtag symbol
    text = re.sub(r'https?:\/\/\S+', '',text) #remove hyperlink
    text = re.sub(r'\n','',text) # remove \n 
    text = re.sub(r'\r','',text) # remove \r
    text = re.sub(r'\\W+', ' ', text) #remove special characters\n",
    return text

for i in range(len(train_data)):
    for j in range(len(train_data[i])):
        train_data[i][j] = clean_text(train_data[i][j]).lower()
        
for i in range(len(dev_data)):
    for j in range(len(dev_data[i])):
        dev_data[i][j] = clean_text(dev_data[i][j]).lower()

In [54]:
train_l = []  # Segmented Text
label_l = []  # Label of Each Text
train_index=[]
for i in range(len(train_data)):
    for tweet in train_data[i]:
        train_l.append(tweet)
        label_l.append(train_labels[i])
        train_index.append(i)

In [55]:
len(train_index)

21764

In [57]:
train_df = pd.DataFrame({'text':train_l, 'label':label_l,'index':train_index})
train_df['label'] = LabelEncoder().fit_transform(train_df['label'])
train_df.shape

(21764, 3)

In [76]:
nan_value = float("NaN")

train_df.replace("", nan_value, inplace=True)

train_df.dropna(axis=0,inplace=True)
train_df

,text,label,index
0,5. can regularly rinsing your nose with saline...,0,0
1,4. can eating garlic help prevent infection wi...,0,0
2,6. do vaccines against pneumonia protect you a...,0,0
3,7. can spraying alcohol or chlorine all over y...,0,0
4,8. how effective are thermal scanners in detec...,0,0
...,...,...,...
21759,all good. just been working out the new home ...,0,1894
21760,best timetable i've heard about! apart from '...,0,1894
21761,and this exactly why you need to come to my c...,0,1894
21762,i might have sent that twice by accident,0,1894


In [77]:
train_df.to_csv('train_bert_seperate.tsv', sep='\t',index=False)

In [61]:
dev_l = []  # Segmented Text
dev_label_l = []  # Label of Each Text
dev_index = []
for i in range(len(dev_data)):
    for tweet in dev_data[i]:
        dev_l.append(tweet)
        dev_label_l.append(dev_labels[i])
        dev_index.append(i)

In [62]:
dev_df = pd.DataFrame({'text':dev_l, 'label':dev_label_l,'index':dev_index})
dev_df['label'] = LabelEncoder().fit_transform(dev_df['label'])
dev_df.shape

(7560, 3)

In [64]:
nan_value = float("NaN")

dev_df.replace("", nan_value, inplace=True)

dev_df.dropna(axis=0,inplace=True)
dev_df

,text,label,index
0,covid-19 fact:are hand dryers effective in kil...,0,0
1,"they are, in fact, germ-breeding factories.",0,0
2,when can we expect the result of my husband's...,0,1
3,_lynne hi have you had any luck? in the same ...,0,1
4,_child eventually..asked for a refund.,0,1
...,...,...,...
7555,exposing yourself to the sun or to temperature...,0,631
7556,just as fast as the virus has spread so has th...,0,631
7557,5g mobile networks do not spread covid-19virus...,0,631
7558,the new coronavirus cannot be transmitted thro...,0,631


In [65]:
dev_df.to_csv('dev_bert_seperate.tsv', sep='\t',index=False)

# Training

In [3]:
!pip install torch torchvision transformers

     |████████████████████████████████| 4.0 MB 7.0 MB/s 
     |████████████████████████████████| 880 kB 28.2 MB/s 
     |████████████████████████████████| 596 kB 58.0 MB/s 
     |████████████████████████████████| 77 kB 8.4 MB/s 
     |████████████████████████████████| 6.6 MB 50.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=27eaf90d32b72170a539a3f805d7a46fb62d0b9c165cef94d54b97956254f5a6
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
from transformers import BertModel

bert_model = BertModel.from_pretrained('bert-base-uncased')

print("Done loading BERT model.")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Done loading BERT model.


In [5]:
from transformers import BertTokenizer

#load BERT's WordPiece tokenisation model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [11]:
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer
import pandas as pd

class SSTDataset(Dataset):

    def __init__(self, filename, maxlen):

        #Store the contents of the file in a pandas dataframe
        self.df = pd.read_csv(filename, delimiter = '\t')

        #Initialize the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        self.maxlen = maxlen

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        #Selecting the sentence and label at the specified index in the data frame
        sentence = self.df.loc[index, 'text']
        label = self.df.loc[index, 'label']

        #Preprocessing the text to be suitable for BERT
        tokens = self.tokenizer.tokenize(sentence) #Tokenize the sentence
        tokens = ['[CLS]'] + tokens + ['[SEP]'] #Insering the CLS and SEP token in the beginning and end of the sentence
        if len(tokens) < self.maxlen:
            tokens = tokens + ['[PAD]' for _ in range(self.maxlen - len(tokens))] #Padding sentences
        else:
            tokens = tokens[:self.maxlen-1] + ['[SEP]'] #Prunning the list to be of specified max length

        tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens) #Obtaining the indices of the tokens in the BERT Vocabulary
        tokens_ids_tensor = torch.tensor(tokens_ids) #Converting the list to a pytorch tensor

        #Obtaining the attention mask i.e a tensor containing 1s for no padded tokens and 0s for padded ones
        attn_mask = (tokens_ids_tensor != 0).long()

        return tokens_ids_tensor, attn_mask, label

In [59]:
from torch.utils.data import DataLoader

#Creating instances of training and development set
#maxlen sets the maximum length a sentence can have
#any sentence longer than this length is truncated to the maxlen size
train_set = SSTDataset(filename = 'train_bert_seperate.tsv', maxlen = 200)
dev_set = SSTDataset(filename = 'dev_bert_seperate.tsv', maxlen = 200)

#Creating intsances of training and development dataloaders
train_loader = DataLoader(train_set,batch_size = 32, num_workers = 2)
dev_loader = DataLoader(dev_set, batch_size = 32, num_workers = 2)

print("Done preprocessing training and development data.")

Done preprocessing training and development data.


In [32]:
import torch
import torch.nn as nn
from transformers import BertModel

class SentimentClassifier(nn.Module):

    def __init__(self):
        super(SentimentClassifier, self).__init__()
        #Instantiating BERT model object 
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')
        
        #Classification layer
        #input dimension is 768 because [CLS] embedding has a dimension of 768
        #output dimension is 1 because we're working with a binary classification problem
        self.cls_layer = nn.Linear(768, 1)

    def forward(self, seq, attn_masks):
        '''
        Inputs:
            -seq : Tensor of shape [B, T] containing token ids of sequences
            -attn_masks : Tensor of shape [B, T] containing attention masks to be used to avoid contibution of PAD tokens
        '''

        #Feeding the input to BERT model to obtain contextualized representations
        outputs = self.bert_layer(seq, attention_mask = attn_masks, return_dict=True)
        cont_reps = outputs.last_hidden_state

        #Obtaining the representation of [CLS] head (the first token)
        cls_rep = cont_reps[:, 0]

        #Feeding cls_rep to the classifier layer
        logits = self.cls_layer(cls_rep)

        return logits

In [33]:
gpu = 0 #gpu ID

print("Creating the sentiment classifier, initialised with pretrained BERT-BASE parameters...")
net = SentimentClassifier()
net.cuda(gpu) #Enable gpu support for the model
print("Done creating the sentiment classifier.")

Creating the sentiment classifier, initialised with pretrained BERT-BASE parameters...


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Done creating the sentiment classifier.


In [34]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
opti = optim.Adam(net.parameters(), lr = 2e-5)

In [35]:
def get_accuracy_from_logits(logits, labels):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    soft_probs = (probs > 0.5).long()
    acc = (soft_probs.squeeze() == labels).float().mean()
    return acc

def evaluate(net, criterion, dataloader, gpu):
    net.eval()

    mean_acc, mean_loss = 0, 0
    count = 0

    with torch.no_grad():
        for seq, attn_masks, labels in dataloader:
            seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)
            logits = net(seq, attn_masks)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            mean_acc += get_accuracy_from_logits(logits, labels)
            count += 1

    return mean_acc / count, mean_loss / count

In [49]:
import time

def train(net, criterion, opti, train_loader, dev_loader, max_eps, gpu):

    best_acc = 0
    st = time.time()
    for ep in range(max_eps):
        
        net.train()
        for it, (seq, attn_masks, labels) in enumerate(train_loader):
            #Clear gradients
            opti.zero_grad()  
            #Converting these to cuda tensors
            seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)

            #Obtaining the logits from the model
            logits = net(seq, attn_masks)

            #Computing loss
            loss = criterion(logits.squeeze(-1), labels.float())

            #Backpropagating the gradients
            loss.backward()

            #Optimization step
            opti.step()
              
            if it % 100 == 0:
                
                acc = get_accuracy_from_logits(logits, labels)
                print("Iteration {} of epoch {} complete. Loss: {}; Accuracy: {}; Time taken (s): {}".format(it, ep, loss.item(), acc, (time.time()-st)))
                st = time.time()

        
        dev_acc, dev_loss = evaluate(net, criterion, dev_loader, gpu)
        print("Epoch {} complete! Development Accuracy: {}; Development Loss: {}".format(ep, dev_acc, dev_loss))
        if dev_acc > best_acc:
            print("Best development accuracy improved from {} to {}, saving model...".format(best_acc, dev_acc))
            best_acc = dev_acc
            torch.save(net.state_dict(), 'sstcls_{}.dat'.format(ep))

In [60]:
num_epoch = 1

#fine-tune the model
train(net, criterion, opti, train_loader, dev_loader, num_epoch, gpu)

Iteration 0 of epoch 0 complete. Loss: 0.774770200252533; Accuracy: 0.5; Time taken (s): 1.1366405487060547
Iteration 100 of epoch 0 complete. Loss: 0.839119017124176; Accuracy: 0.125; Time taken (s): 96.53734254837036
Iteration 200 of epoch 0 complete. Loss: 0.577962338924408; Accuracy: 0.5625; Time taken (s): 100.59058785438538
Iteration 300 of epoch 0 complete. Loss: 0.8304238319396973; Accuracy: 0.21875; Time taken (s): 102.67964887619019
Iteration 400 of epoch 0 complete. Loss: 0.6861435770988464; Accuracy: 0.5; Time taken (s): 103.10173916816711
Iteration 500 of epoch 0 complete. Loss: 0.42616337537765503; Accuracy: 0.71875; Time taken (s): 102.88800501823425
Iteration 600 of epoch 0 complete. Loss: 0.6111243963241577; Accuracy: 0.65625; Time taken (s): 103.05852913856506
Epoch 0 complete! Development Accuracy: 0.7447034120559692; Development Loss: 0.4764299577938677
Best development accuracy improved from 0 to 0.7447034120559692, saving model...


# Testing

In [66]:
f = open(f'./test_tweet_text.pckl','rb')
test_data = pickle.load(f)
f.close()

In [67]:
for i in range(len(test_data)):
    for j in range(len(test_data[i])):
        test_data[i][j] = clean_text(test_data[i][j]).lower()

## not sure what to do now so get the merge as the test input

In [17]:
test_merge_events=[]
for event in test_data:
    merge = ''
    for tweet in event:
        merge = merge + tweet
    test_merge_events.append(merge)

In [68]:
test_l = []  # Segmented Text  # Label of Each Text
test_index = []
for i in range(len(test_data)):
    for tweet in dev_data[i]:
        test_l.append(tweet)
        test_index.append(i)

ERROR! Session/line number was not unique in database. History logging moved to new session 587


In [69]:
test_df = pd.DataFrame({'text':test_l,'index': test_index})

In [70]:
test_df

,text,index
0,covid-19 fact:are hand dryers effective in kil...,0
1,"they are, in fact, germ-breeding factories.",0
2,when can we expect the result of my husband's...,1
3,_lynne hi have you had any luck? in the same ...,1
4,_child eventually..asked for a refund.,1
...,...,...
6699,when i was a little girl i remember hearing h...,557
6700,“: man who portrayed the marlboro man in the 1...,557
6701,this is an example of irony. not coincidence.,557
6702,he was only 72 😜rt man who portrayed the marl...,557


In [71]:
nan_value = float("NaN")

test_df.replace("", nan_value, inplace=True)

test_df.dropna(axis=0 ,inplace=True)

In [72]:
test_df

,text,index
0,covid-19 fact:are hand dryers effective in kil...,0
1,"they are, in fact, germ-breeding factories.",0
2,when can we expect the result of my husband's...,1
3,_lynne hi have you had any luck? in the same ...,1
4,_child eventually..asked for a refund.,1
...,...,...
6699,when i was a little girl i remember hearing h...,557
6700,“: man who portrayed the marlboro man in the 1...,557
6701,this is an example of irony. not coincidence.,557
6702,he was only 72 😜rt man who portrayed the marl...,557


In [73]:
test_df.to_csv('test_bert_seperate.tsv', sep='\t',index=False)

In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer
import pandas as pd

class TestDataset(Dataset):

    def __init__(self, filename, maxlen):

        #Store the contents of the file in a pandas dataframe
        self.df = pd.read_csv(filename, delimiter = '\t')

        #Initialize the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        self.maxlen = maxlen

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        #Selecting the sentence and label at the specified index in the data frame
        sentence = self.df.loc[index, 'sentence']

        #Preprocessing the text to be suitable for BERT
        tokens = self.tokenizer.tokenize(sentence) #Tokenize the sentence
        tokens = ['[CLS]'] + tokens + ['[SEP]'] #Insering the CLS and SEP token in the beginning and end of the sentence
        if len(tokens) < self.maxlen:
            tokens = tokens + ['[PAD]' for _ in range(self.maxlen - len(tokens))] #Padding sentences
        else:
            tokens = tokens[:self.maxlen-1] + ['[SEP]'] #Prunning the list to be of specified max length

        tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens) #Obtaining the indices of the tokens in the BERT Vocabulary
        tokens_ids_tensor = torch.tensor(tokens_ids) #Converting the list to a pytorch tensor

        #Obtaining the attention mask i.e a tensor containing 1s for no padded tokens and 0s for padded ones
        attn_mask = (tokens_ids_tensor != 0).long()

        return tokens_ids_tensor, attn_mask

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\Anaconda\envs\CV\lib\site-packages\transformers\utils\import_utils.py", line 857, in _get_module
    return importlib.import_module("." + module_name, self.__name__)
  File "D:\Anaconda\envs\CV\lib\importlib\__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1006, in _gcd_import
  File "<frozen importlib._bootstrap>", line 983, in _find_and_load
  File "<frozen importlib._bootstrap>", line 967, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 677, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "D:\Anaconda\envs\CV\lib\site-packages\transformers\onnx\config.py", line 35, in <module>
    from PIL import Image
  File "D:\Anaconda\envs\CV\lib\site-packages\PIL\Image.py", line 50, in <module>
    

In [70]:
test_set = TestDataset(filename = 'test_bert_seperate.tsv', maxlen = 200)

#Creating intsances of training and development dataloaders
test_loader = DataLoader(test_set,batch_size = 1, num_workers = 0)

In [71]:
def predict(net, test_loader,weight_file):
    # load weight
    net.load_state_dict(torch.load(weight_file))
    
    predictions = []
    
    # Predict process
    with torch.no_grad():
        for seq, attn_masks in test_loader:
            seq, attn_masks = seq.cuda(gpu), attn_masks.cuda(gpu)
            logits = net(seq, attn_masks)
            probs = torch.sigmoid(logits.unsqueeze(-1))
            soft_probs = (probs > 0.5).long()
            predictions.append(soft_probs.cpu().numpy().squeeze())
    return predictions

In [72]:
weight_file = "sstcls_0.dat"
prediction = predict(net, test_loader,weight_file)

In [74]:
df = pd.DataFrame({"Id": range(len(prediction)),"Predicted": prediction}) 
df.to_csv('bert_test_seperate.csv',index=False)

### LSTM
model the source tweet and replies as a sequence of tweets using recurrent networks